<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">Thierry Parmentelat &amp; Arnaud Legout&nbsp;<img src="media/both-logos-small-alpha.png" style="display:inline"></span><br/>

# la librairie `asyncio`

* boucle d'événements

* synchronisation: `Queue`, `Lock` et `Semaphore`

* interaction avec les processus:
  * package `asyncio.subprocess`

* `Transport` et `Protocol` 
* `Streams`

In [ ]:
import asyncio

from asynchelpers import start_timer, show_timer

# synchronisation avec une queue

In [ ]:
asyncio.Queue?

In [ ]:
queue = asyncio.Queue(maxsize=1)

In [ ]:
async def producer(queue):
    count = 1
    while True:
        await queue.put(f'tick{count}')
        count += 1
        await asyncio.sleep(1)

In [ ]:
async def consumer(queue):
    while True:
        received = await queue.get()
        show_timer(f"got {received}")
        

In [ ]:
# on ajoute les coroutines dans la boucle
asyncio.ensure_future(producer(queue))
asyncio.ensure_future(consumer(queue))

In [ ]:
start_timer()

# interrompre avec la touche 'i' 
# plusieurs fois si nécessaire

try:
    asyncio.get_event_loop().run_forever()
except KeyboardInterrupt as e:
    print("bye")

# limiter le parallèlisme avec `Queue`

In [ ]:
asyncio.set_event_loop(asyncio.new_event_loop())

In [ ]:
window = asyncio.Queue(maxsize = 4)

In [ ]:
async def job(i):
    # prendre un jeton dans la queue
    await window.put(None)
    # pas tout le monde la même durée
    duration = (i % 3) + 1
    message = f"job{i} - duration {duration}"
    show_timer(">>>", message)
    await asyncio.sleep(duration)
    show_timer("<<<", message)
    # libérer le jeton
    await window.get()

In [ ]:
for i in range(8):
    asyncio.ensure_future(job(i))

start_timer()
try:
    asyncio.get_event_loop().run_forever()
except:
    print('bye')

### Séquencement des jobs

|     | j0 (1) | j1(2) | j2(3) | j3(1) | j4(2) | j5(3) | j6(1) | j7(2) |
|-----|--------|-------|-------|-------|-------|-------|-------|-------|
| 0-1 | `*`    | `*`   | `*`   | `*`   |       |       |       |       |
| 1-2 |        | `*`   | `*`   |       | `*`   | `*`   |       |       |
| 2-3 |        |       | `*`   |       | `*`   | `*`   | `*`   |       |
| 3-4 |        |       |       |       |       | `*`   |       | `*`   |
| 4-5 |        |       |       |       |       |       |       | `*`   |

# valeur ajoutée

* boucle d'événements complice avec:
  * `sleep()`,
  * `subprocess.*`
  * les classes abstraites:
  
    `Transport`, `Protocol`, `Stream`

* tire profit de l'OS
  * *signal()* : interruptions
  * *select()* : événements liés aux entrée-sorties
  * ...

# usage

* commencer avec les librairies de haut niveau
  * HTTP (`aoihttp`) 
  * ssh (`asyncssh`)
  * telnet (`telnetlib3`) 
  * BdD's - ...